# FAIR USE INDEX
### Problema
Cada vez más los Datos y Algoritmos están presentes en los sistemas y/o dispositivos que influyen en la vida de las personas. Entendiendo a la persona como un individuo de la especie humana. 

En estos tiempos nos enfrentamos a un escenario caracterizado por diversas oportunidades, tensiones y desafíos que demandan compromisos a nivel individual, colectivo y ambiental. Los estados, empresas, organismos, u otras organizaciones habilitan espacios de reflexión y acción ante el impacto del uso de Datos, Ciencias de Datos e Inteligencia Artificial (IA) en las personas y en la naturaleza.

¿Pueden las IA ser consideradas “autores/as” o “titulares de derechos”?. Para empezar a responder esta pregunta, de debate y reflexiòn en estos años, podemos empezar por investigar sobre la ley de derecho de autor y regulaciones en IA. 

### Objetivo
Identificar caracterìsticas de los casos judiciales que los tribunales de EE.UU han determinado previamente como justos o no justos en relaciòn a los principios y a la aplicación del uso legítimo de la ley de derechos de autor. 
EE.UU es un paìs con antecedentes controversiales en relaciòn al cumplimiento y la flexibilidad de la ley de derecho de autor aplicada en empresas tecnològicas. Preguntas a considerar:
* ¿Cuàles son los tipos de casos judiciales determinados por los tribunales de EE.UU?
* ¿Cuàles son las categorìas de casos judiciales exploradas por los tribunales de EE.UU ?

El Índice de Uso Justo  (Fair Use Index) rastrea una variedad de decisiones judiciales para ayudar a abogados y no abogados a comprender mejor los tipos de usos que los tribunales han determinado previamente como justos o no justos. Las decisiones abarcan múltiples jurisdicciones federales, incluida la Corte Suprema de Estados Unidos, los tribunales de apelación de circuito y los tribunales de distrito. Tenga en cuenta que, si bien el Índice incorpora una amplia selección de casos, no incluye todas las opiniones judiciales sobre uso legítimo. 

### Dataset
Los datos que provienen del Índice de Uso Justo de la Oficina de Derechos de Autor de EE.UU.  
Dos archivos son considerados fair_use_cases.csv y fair_use_findings.csv. 
* Fuente de datos: [link](https://github.com/rfordatascience/tidytuesday/blob/master/data/2023/2023-08-29/readme.md)
* Dominio de datos: [link](https://www.copyright.gov/fair-use/index.html)

#### Diccionario de datos

**fair_use_cases**
* case: 	character 	The name and number of the case.
* year: 	integer 	The year in which the case was decided.
* court: 	character 	The court in which the ruling was made.
* jurisdiction: 	character 	The jurisdiction of that court.
* categories: 	character 	A comma- or semicolon-separated list of categories to which the case belongs. These have not been normalized.
* outcome: 	character 	A string describing the outcome of the case.
* fair_use_found :	logical 	Whether fair use was found by the court. FALSE might sometimes indicate a more complicated finding.

**fair_use_findings**
* title 	character 	The title of the case.
* case_number 	character 	The case number or numbers of the case.
* year 	character 	The year in which the finding was made (or findings were made).
* court 	character 	The court or courts involved.
* key_facts 	character 	The key facts of the case.
* issue 	character 	A brief description of the fair use issue.
* holding 	character 	The decision of the court in paragraph form.
* tags 	character 	Comma- or semicolon-separated tags for this case.
* outcome 	character 	A brief description of the outcome of the case. These fields have not been normalized.



In [1]:
# Importamos librerias
import pandas as pd
pd.set_option('display.max_colwidth', None)
import mysql.connector
from sqlalchemy import create_engine
import pymysql

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
# Leemos archivos extraidos Tidy Tuesday
fair_use_cases_df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2023/2023-08-29/fair_use_cases.csv')
fair_use_findings_df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2023/2023-08-29/fair_use_findings.csv')


In [3]:
print('Archivo fair_use_cases \n')
print(fair_use_cases_df.info())
fair_use_cases_df.head()

Archivo fair_use_cases 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   case            251 non-null    object
 1   year            251 non-null    int64 
 2   court           251 non-null    object
 3   jurisdiction    251 non-null    object
 4   categories      251 non-null    object
 5   outcome         251 non-null    object
 6   fair_use_found  251 non-null    bool  
dtypes: bool(1), int64(1), object(5)
memory usage: 12.1+ KB
None


,case,year,court,jurisdiction,categories,outcome,fair_use_found
0,"De Fontbrune v. Wofsy, 39 F.4th 1214 (9th Cir. 2022)",2022,9th Circuit,9th Circuit,Education/Scholarship/Research; Photograph,Fair use not found,False
1,"Sedlik v. Von Drachenberg, No. CV 21-1102 (C.D. Cal. May 31, 2022)",2022,C.D. Cal.,9th Circuit,Painting/Drawing/Graphic; Photograph,Preliminary finding; Fair use not found,False
2,"Sketchworks Indus. Strength Comedy, Inc. v. Jacobs, No. 19-CV-7470-LTS-VF (S.D.N.Y. May 12, 2022)",2022,S.D.N.Y.,2nd Circuit,Film/Audiovisual; Music; Parody/Satire; Review/Commentary,Fair use found,True
3,"Am. Soc'y for Testing & Materials v. Public.Resource.Org, Inc., No. 13-cv-1215 (D.D.C. Mar. 31, 2022)",2022,D.D.C.,District of Columbia Circuit,Education/Scholarship/Research; Textual Work; Used in government proceeding,Mixed Result,False
4,"Yang v. Mic Network Inc., Nos. 20-4097-cv, 20-4201-cv (2d Cir. Mar. 29, 2022)",2022,2d Circuit,2nd Circuit,News reporting; Photography,Fair use found,True


In [4]:
fair_use_cases_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
case,251,251,"De Fontbrune v. Wofsy, 39 F.4th 1214 (9th Cir. 2022)",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
year,251.0,NaN,NaN,NaN,2003.697211,18.226902,1841.0,1993.5,2009.0,2017.0,2022.0
court,251,53,S.D.N.Y.,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jurisdiction,251,14,2nd Circuit,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categories,251,155,Education/Scholarship/Research; Textual work,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
outcome,251,16,Fair use not found,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fair_use_found,251,2,False,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print('Archivo fair_use_findings \n')
print(fair_use_findings_df.info())
fair_use_findings_df.head()

Archivo fair_use_findings 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        251 non-null    object
 1   case_number  251 non-null    object
 2   year         251 non-null    object
 3   court        251 non-null    object
 4   key_facts    251 non-null    object
 5   issue        251 non-null    object
 6   holding      251 non-null    object
 7   tags         251 non-null    object
 8   outcome      251 non-null    object
dtypes: object(9)
memory usage: 17.8+ KB
None


title  \
0                                           De Fontbrune v. Wofsy   
1                                       Sedlik v. Von Drachenberg   
2              Sketchworks Indus. Strength Comedy, Inc. v. Jacobs   
3  Am. Soc'y for Testing & Materials v. Public.Resource.Org, Inc.   
4                                        Yang v. Mic Network Inc.   

                                                                               case_number  \
0                                                            39 F.4th 1214 (9th Cir. 2022)   
1                                 No. CV 21-1102, 2022 WL 2784818 (C.D. Cal. May 31, 2022)   
2               No. 19-CV-7470-LTS-VF, 2022 U.S. Dist. LEXIS 86331 (S.D.N.Y. May 12, 2022)   
3                 No. 13-cv-1215 (TSC), 2022 U.S. Dist. LEXIS 60922 (D.D.C. Mar. 31, 2022)   
4  Nos. 20-4097-cv(L), 20-4201-cv (XAP), 2022 U.S. App. LEXIS 8195 (2d Cir. Mar. 29, 2022)   

   year                                                               court  \
0  2022                United States Court of Appeals for the Ninth Circuit   
1  2022  United States District Court for the Southern District of New York   
2  2022  United States District Court for the Southern District of New York   
3  2022           United States District Court for the District of Columbia   
4  2022               United States Court of Appeals for the Second Circuit   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    key_facts  \
0                                                                                                                                                                                                                                                        Plaintiffs own the rights to a catalogue comprised of 16,000 photographs of Pablo Picasso’s work, which was originally compiled by Picasso’s friend in 1932 (the “Zervos Catalogue”). In 1995, after obtaining permission from Picasso’s estate to publish a work illustrating and describing works by Picasso, Defendants Alan Wofsy and his company Alan Wofsy & Associates began publishing The Picasso Project—–a series of volumes reproducing images of Picasso’s work, including 1,492 photographs from the Zervos Catalogue. Plaintiffs sued for copyright infringement. A French court held the photographs were protected by copyright because they “added creative features through deliberate choices of lighting, the lens, filters, [

In [6]:
fair_use_findings_df.describe(include='all').T

,count,unique,top,freq
title,251,248,"Bouchat v. Balt. Ravens Ltd. P’ship,",2
case_number,251,251,39 F.4th 1214 (9th Cir. 2022),1
year,251,56,2020,17
court,251,60,United States District Court for the Southern District of New York,48
key_facts,251,251,"Plaintiffs own the rights to a catalogue comprised of 16,000 photographs of Pablo Picasso’s work, which was originally compiled by Picasso’s friend in 1932 (the “Zervos Catalogue”). In 1995, after obtaining permission from Picasso’s estate to publish a work illustrating and describing works by Picasso, Defendants Alan Wofsy and his company Alan Wofsy & Associates began publishing The Picasso Project—–a series of volumes reproducing images of Picasso’s work, including 1,492 photographs from the Zervos Catalogue. Plaintiffs sued for copyright infringement. A French court held the photographs were protected by copyright because they “added creative features through deliberate choices of lighting, the lens, filters, [and] framing or angle of view.” In 2001, Plaintiffs obtained a judgment in France that subjected Defendants to damages for any further acts of infringement. In 2012, after discovering copies of The Picasso Project in a French bookstore, Plaintiffs enforced their judgment in France and were awarded €2 million. Plaintiffs sought recognition of the judgment in the U.S. courts. The district court granted summary judgment for Defendants, determining that the French judgment was “repugnant to U.S. public policy protecting free expression” because it failed to provide a fair use defense. Plaintiffs appealed; and Defendants cross-appealed on other defenses.",1
issue,251,248,Whether a university’s electronic distribution of unlicensed copyrighted works to students is a fair use.,2
holding,251,251,"The panel held that the first factor, the purpose and character of the use, weighed against fair use because Defendants conceded that The Picasso Project was a commercial venture and the use at issue—reproduction of the photographs in a book illustrating Picasso’s works—was not transformative. Specifically, the court noted that Defendants’ use “did not serve an ‘entirely different function’ than the originals,” but had overlapping purposes, and the insertion of informative captions did not “necessarily” transform the works. The second factor, the nature of the copyrighted work, did not favor fair use because, although the works were published and documentary in nature, the French court had concluded that the photographs exhibited creative elements. The court determined that the third factor, the amount and substantiality of the work used, weighed against fair use because Defendants failed to demonstrate that “copying the entirety of each photograph was necessary.” The fourth factor, the effect of the use upon the potential market for or value of the copyrighted work, also weighed against fair use because there is a presumption of market harm when the use is commercial and non-transformative. Although Defendants presented evidence that auction prices for the Zervos Catalogue increased while The Picasso Project was on the market, Defendants had not provided evidence that “widespread appropriation” of the works would not harm the market for the photographs. Weighing all the factors, the court had “serious doubts” that fair use would protect Defendants’ use, and, accordingly, granted summary judgment to Plaintiffs on the public policy defense.",1
tags,251,207,Second Circuit; Education/Scholarship/Research; Textual work,4
outcome,251,22,Fair use not found,99


Mediante la lectura de los archivos, se observan 251 registros en cada archivo. Se observa diferencias en proporciones en la variable outcome. Por lo que hay inconsistencia que se deberà analizar para relacionar los archivos.
* **fair_use_cases_df**: presenta 251 casos judiciales que corresponden 1841 a 2022. 14 juridicciones y 53 cortes. 155 categorìas que no estàn normalizadas.
* **fair_use_findings_df**: presenta 251 casos judiciales. 60 cortes. 22 outcome que no estàn normalizados.

Estos archivos se guardaràn en una base de datos mysql, para disponer de un dataset màs limpio.

#### 1- Cargamos los archivos a base de datos Mysql

In [7]:
# Creamos objeto de conexion a base de datos
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "Ejercicio2024!"
)
 
print(mydb)

In [8]:
# Funciones ùtiles para trabajar con base de datos
def create_database(cursor,DB_NAME):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
        
def read_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute( query )
        names = [ x[0] for x in cursor.description]
        rows = cursor.fetchall()
        return pd.DataFrame( rows, columns=names)
    finally:
        if cursor is not None:
            cursor.close()

In [9]:
# Creamos base de datos
mycursor = mydb.cursor()
create_database(mycursor, "Fair_use")

Failed creating database: 1007 (HY000): Can't create database 'Fair_use'; database exists


In [10]:
# Llamamos a la base de datos
mycursor.execute("USE Fair_use")

In [11]:
mycursor.execute("DROP TABLE `fair_use_cases`")

In [12]:
# Creamos tabla para fair_use_cases` con la que estaremos màs trabajando
mycursor.execute("CREATE TABLE `fair_use_cases` (`id` INT AUTO_INCREMENT PRIMARY KEY, `case` VARCHAR(255), `year` SMALLINT(255), `court` VARCHAR(255) ,`jurisdiction` VARCHAR(255), `categories` VARCHAR(255), `outcome` VARCHAR(255), `fair_use_found` VARCHAR(255))ENGINE=InnoDB")


In [14]:
# Datos de conexiòn
hostname= "localhost"
database= "Fair_use"
username= "root"
password= "Ejercicio2024!"
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=database, user=username, pw=password))

In [15]:
# Insertamos registros a fair_use_cases
fair_use_cases_df.to_sql('fair_use_cases', con=engine, if_exists='append', index=False)
# Creamos tabla, que hay que normalizar fair_use_findings
fair_use_findings_df.to_sql('fair_use_findings', con=engine, if_exists='replace', index=False)



251

#### 2 - Primeras exploraciòn de los datos

**¿Cuàles son los tipos de casos judiciales determinados por los tribunales de EE.UU?**

Se analiza la variable fair_use_found y outcome.

In [28]:
# Generamos query 
query = """
SELECT  fair_use_found, count(*) Count 
FROM fair_use_cases 
GROUP BY fair_use_found
ORDER BY COUNT DESC
"""
df = read_query(mydb, query)
df

,fair_use_found,Count
0,0,150
1,1,101


In [19]:
# Gràfico para fair_use_found
fig = px.pie(df, values='count', names='fair_use_found')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(title_text='Porcentaje de casos judiciales segùn "uso legitimo" (fair use found) determinado por los tribunales')
fig.show()


In [24]:
# llamamos a la tabla fair_use_case y la convertimos en dataframe para trabajar màs comodos
query = """
SELECT *
FROM fair_use_cases 
"""
fair_use_cases_df = read_query(mydb, query)

In [25]:
# Histograma de años
ig = px.histogram(fair_use_cases_df, x="year", color="fair_use_found")
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(title_text='Distribuciòn de casos judiciales segùn año y "uso legitimo" (fair use found) determinado por los tribunales')
fig.show()


In [26]:
# Gràfico de barra para outcome
query = """
SELECT outcome, count(*) count
FROM fair_use_cases 
GROUP BY outcome
ORDER BY COUNT DESC
"""

fig = px.bar(read_query(mydb, query),x='outcome', y='count')
fig.update_layout(title_text='Cantidad de casos judiciales segùn descripciòn determinada (outcome) por los tribunales')
fig.show()

De los 251 casos judiciales de `fair_use_cases`
* 40,2% casos fueron determinados de uso legitimo por los tribunales entre 1841 a 2022.
* Desde el 2010 se observa màs casos judiciales determinados de uso legitimos. 
* Hay que normalizar la variable outcome.

**¿Cuàles son las categorìas de casos judiciales explorados por los tribunales de EE.UU ?**

Este campo no esta normalizado por ello se identifican 155 categorìas. Vamos a identificar los Top categorias y personalizar en las  categorìas relacionadas a Internet/Digitization.

In [29]:
# Exploramos el dataframe asociado a fair_use_case
fair_use_cases_df['categories'].unique()

array(['Education/Scholarship/Research; Photograph',
       'Painting/Drawing/Graphic; Photograph',
       'Film/Audiovisual; Music; Parody/Satire; Review/Commentary',
       'Education/Scholarship/Research; Textual Work; Used in government proceeding',
       'News reporting; Photography',
       'Painting/Drawing/Graphic; Parody/Satire',
       'Internet/Digitization; News Reporting; Photograph',
       'Internet/Digitization; Textual Work',
       'Educational/Scholarship/Research; Internet/Digitization; Textual Work',
       'Internet/Digitization; Photograph; Review/Commentary',
       'Parody/Satire; Review/Commentary; Sculpture; Painting/Drawing/Graphic',
       'Painting/Drawing/Graphic; Photograph; Unpublished',
       'Film/Audiovisual; Internet/Digitization; Parody/Satire; Photograph; Sculpture',
       'Internet/Digitization; Painting/Drawing/Graphic',
       'Music; Film/Audiovisual', 'Computer Program',
       'Education/Scholarship/Research; Internet/Digitization; Photog

In [30]:
# Calculamos el promedio de casos judiciales para identificar los Top categorias
categories_df = fair_use_cases_df['categories'].value_counts().reset_index()
print("Cantidad promedia de casos judiciales por categorìa:", categories_df['count'].mean())


Cantidad promedia de casos judiciales por categorìa: 1.6193548387096774


In [31]:
print("Categorias que superan la cantidad promedio de casos judiciales por categorìa")
categories_df[categories_df['count'] > categories_df['count'].mean()]

Categorias que superan la cantidad promedio de casos judiciales por categorìa


,categories,count
0,Education/Scholarship/Research; Textual work,16
1,Textual work,9
2,Film/Audiovisual; Parody/Satire,7
3,Film/Audiovisual; News reporting,6
4,Painting/Drawing/Graphic; Photograph,5
5,Computer program,5
6,Photograph,5
7,Internet/Digitization; Photograph,4
8,Film/Audiovisual,4
9,Painting/Drawing/Graphic,4


In [33]:
# Seleccionamos los casos de Internet/Digitalizaciòn
fair_use_cases_InternetDig_df = fair_use_cases_df.loc[fair_use_cases_df['categories'].str.contains('Internet/Digitization'),]
print("Estadisticas descriptivas de casos judiciales asociados a categorìas que contienen Internet/Digitization ")
fair_use_cases_InternetDig_df.describe(include='all'). T

Estadisticas descriptivas de casos judiciales asociados a categorìas que contienen Internet/Digitization 


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,50.0,NaN,NaN,NaN,67.26,45.951088,7.0,29.5,56.5,92.0,180.0
case,50,50,"McGucken v. Newsweek LLC, 19 Civ. 9617 (KPF) (S.D.N.Y. Mar. 21, 2020)",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
year,50.0,NaN,NaN,NaN,2015.46,6.141894,1996.0,2014.0,2018.0,2020.0,2022.0
court,50,21,S.D.N.Y.,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jurisdiction,50,9,2nd Circuit,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categories,50,39,Internet/Digitization; Photograph,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
outcome,50,10,Fair use found,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fair_use_found,50,2,0,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_gp = fair_use_cases_InternetDig_df.groupby(by = ['fair_use_found','categories',])['fair_use_found'].agg(['count']).reset_index()
fig = px.sunburst(df_gp , path=['fair_use_found', 'categories'], values='count')
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
)

fig.update_layout(title_text='Cantidad de casos judiciales asociados a Internet/Digitization segùn "uso legitimo" (fair use found) determinado por los tribunales')
fig.show()

Relacionando fair_use_cases con fair_use_findings_df. 

Se desea tener màs detalle de los casos judiciales asociado Internet/Digitization

In [38]:
query ="""
SELECT court, fair_use_found_count,
RANK() OVER(ORDER BY fair_use_found_count DESC) as 'rank'
FROM
(
SELECT
  court,
  SUM(fair_use_found) AS fair_use_found_count
FROM fair_use_cases WHERE categories LIKE '%Internet/Digitization%'
GROUP BY court
) as F

"""
df = read_query(mydb, query)
print("Ranking de Cortes-Tribunales que determinan judiciales con uso legitimo asociados a 'Internet/Digitization")
df

Ranking de Cortes-Tribunales que determinan judiciales con uso legitimo asociados a 'Internet/Digitization


,court,fair_use_found_count,rank
0,2d Circuit,3.0,1
1,C.D. Cal.,2.0,2
2,S.D.N.Y.,2.0,2
3,D. Nev.,2.0,2
4,E.D. Va.,2.0,2
5,5th Circuit,1.0,6
6,E.D.N.Y.,1.0,6
7,S.D. Ohio,1.0,6
8,4th Circuit,1.0,6
9,N.D. Ill.,1.0,6


In [39]:
# Seleccionamos los casos de Internet/Digitalizaciòn que machean entre las tablas fair_use_cases y fair_use_findings
query = """
SELECT F.*,FF.* 
FROM (SELECT * FROM fair_use_cases  WHERE categories LIKE "%Internet/Digitization%") AS F 
INNER JOIN (SELECT *, IF(title LIKE "%,%", CONCAT(title," ",case_number), CONCAT(title,", ",case_number)) "case" FROM fair_use_findings) AS FF
ON F.case = FF.case
"""

In [43]:
# Se integran solo 50 registros, hay que limpiar campos title y case_number
df = read_query(mydb, query)
df[['title','case_number','categories','fair_use_found','key_facts','issue']]

,title,case_number,categories,fair_use_found,key_facts,issue
0,Monsarrat v. Newman,28 F.4th 314 (1st Cir. 2022),Internet/Digitization; Textual Work,1,"In 2010, Plaintiff Jonathan Monsarrat created a post on a community forum discussion thread hosted on the Russian-owned social networking platform, LiveJournal. Monsarrat’s post responded to allegedly defamatory posts about him on the thread by threatening to report the other users for violations of LiveJournal’s harassment policy, which he quoted and linked to in the post. Monsarrat subsequently registered his post with the U.S. Copyright Office in 2012. In 2017, LiveJournal revised its terms of service to comply with Russian law, which permitted censorship. As a result, Defendant Ron Newman moved the forum, including Monsarrat’s registered post, to another platform, Dreamwidth, by copying the forum’s discussion threads and reposting them on Dreamwidth. Monsarrat sued for copyright infringement and Newman moved to dismiss the claim, asserting fair use. The district court held that Newman’s reproduction of the post as part of the discussion thread was fair use. Monsarrat appealed.",Whether copying a post from one social networking platform to another is fair use.
1,Bell v. Eagle Mt. Saginaw Indep. Sch. Distr.,27 F.4th 313 (5th Cir. 2022),Educational/Scholarship/Research; Internet/Digitization; Textual Work,1,"Plaintiff Dr. Keith Bell, a sports psychologist, wrote and published Winning Isn’t Normal, a book that provides strategies for success in athletics. Bell separately registered the copyright for a 230-word excerpt from the book (the “WIN Passage”). Bell sells merchandise that displays the WIN Passage online and licenses its use. In December 2017, Chisholm Trail High School’s softball team and color guard posted the WIN Passage to their Twitter accounts, crediting Bell as the author. In November 2018, Bell notified Defendant Eagle Mountain Saginaw Independent School District that the Twitter posts infringed his copyright. In response, Defendant removed both posts, told Bell that the mistake was a “teachable moment,” and announced a training program to avoid similar incidents. After settlement negotiations failed, Bell brought an action for copyright infringement. Defendant moved to dismiss, asserting that the school’s use was fair. The district court granted Defendant’s motion and awarded it attorney’s fees. Bell appealed.",Whether use of a motivational passage excerpted from a book in social media posts by public school athletics programs constitutes fair use.
2,Marano v. Metro. Museum of Art,844 F. App’x 436 (2d Cir. 2021),Education/Scholarship/Research; Internet/Digitization; Photograph,1,"Plaintiff Lawrence Marano, a professional photographer, took a photograph of Eddie Van Halen playing his “Frankenstein” guitar at a concert (the “Photograph”). The Metropolitan Museum of Art (the “Met”), a nonprofit museum that displays works of art to the public, included the Photograph in an online catalogue for a physical exhibition on rock and roll instruments. The catalogue included 185 webpages representing the exhibit objects on physical display. The Photograph was displayed as a thumbnail image on the webpage discussing the “Frankenstein” guitar, along with a large photograph of the guitar, two thumbnail images, and two paragraphs of text describing the guitar’s history and technical specifications. Online visitors could click on the Photograph’s thumbnail to view a larger version of it. Marano sued the Met for willful copyright infringement, and the district court found that the Met’s use of the Photograph was a fair use. Marano appealed.",Whether posting a photograph in a museum exhibit’s online catalogue to depict an object shown in the Photograph is a fair use.
3,"Fox Broad. Co. v. Dish Network, LLC,","No. 2:12-cv-04529-DMG-SH (C.D. Cal. Jan. 12, 2015)",Film/Audiovisual; Format shifting/Space shifting; Internet/Digitization,0,"Defendant Dish Network, LLC (Dish) developed and 

Està informaciòn detalladas de casos judiciales, nos permite identificar casos en donde hay màs flexibilidad o control de la ley de derecho de autor en entornos digitales en Estados unidos.